In [ ]:
from pyspark.sql.functions import * # pyspark functions

In [ ]:
# file location and type
pin_location = "/mnt/data/topics/0affea73130b.user/partition=0/*.json" 
file_type = "json"
# ask Spark to infer the schema
infer_schema = "true"
# read in json files from mounted s3 bucket
user_df = spark.read.format(file_type).option("inferSchema", infer_schema).load(pin_location)

In [ ]:
cleaned_df = user_df.cache()

In [ ]:
# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# create a new column user_name that concatenates the information found in the first_name and last_name columns
# drop first_name and last_name columns from the dataframe
cleaned_df = cleaned_df.withColumn("user_name", concat("first_name", lit(" "), "last_name"))
cleaned_df = cleaned_df.drop("first_name", "last_name")

# cast int datatype to ind
cleaned_df = cleaned_df.withColumn("ind", cleaned_df["ind"].cast("int"))

# convert the date_joined column from a string to a timestamp data type
cleaned_df = cleaned_df.withColumn("date_joined", to_timestamp("date_joined"))

# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# reorder the dataframe columns to have the following column order:
cleaned_df = cleaned_df.select("ind", "user_name", "age", "date_joined")

In [ ]:
user_df = cleaned_df.cache()